## School project - 5MLDE
The following notebook has been created for a school project model deployment. The subject and the questions are available in the appendix.

The group members who participated in this project are:
- AMIMI Lamine
- BEZIN Théo
- LECOMTE Alexis
- PAWLOWSKI Maxence

### Main index
1. Data analysis
2. **Data quality tests (you are here)**
3. _Appendix_

# 2 - Data quality tests
The purpose of this first notebook will be to explain how to setup the Great Expectations base and configuration to obtain the results available in the /config/great_expectations folder.

### Index
<ol type="A">
  <li>Initialization of the Great Expectation base</li>
  <li>Datasource creation and configuration</li>
  <li>Expectations Suite creation and configuration</li>
  <li>Conclusion</li>
</ol>

## A - Initialization of the Great Expectation base

In the jupyter-server/config folder, we will initiate the Great Expectations (GX) project base with this command :

`great_expectations init`

Then, just press enter.

## B - Datasource creation and configuration

In the newly created folder (named *great_expectations* by default, we will do most of the remaining work.
We need a new datasource for GX to work with. We can load our csv file with this command :

`great_expectations datasource new`

We are processing our data with Pandas, so we will choose "Files on a filesystem (for processing with Pandas or Spark)".
On the next prompt, select "Pandas".
We are asked to enter the path of our data directory. Since we are working from the jupyter container, the data folder path is "jupyter-server/data".

A new notebook will be created in the *uncommitted* folder named "datasource_new". Open this notebook and change the value *my_datasource* to whatever you want your datasource to be called. In this project, we named it "android_malware". Then, execute all cells in the notebook. If successful, the last cell will print the newly created datasource.

## C - Expectations Suite creation and configuration

Next step is to create the Expectation Suite by doing this command :
`great_expectations suite new`

We will use a Data Assistant to import our data file to this suite. So we will choose "Automatically, using a Data Assistant".

Our CSV is now displayed as a choice for our suite, select it and give a name to the new suite. For this project, we named it "android_malware_suite".
A new notebook is now created in the *uncommitted* folder named "edit_android_malware_suite".


The previously created notebook need some little tweaks to match our needs.
By default, all the columns of the dataset are excluded in the "exclude_column_names" array. We only need the "Flow ID" column to be excluded. So we are removing every column from the array except this one.

Finally, we can execute all cells of the suite notebook. This will generate a Data Docs presenting the expectations results

## D - Conclusion ?

The next notebook is the appendix.